In [1]:
%matplotlib inline
import pprint  
import math
import numpy as np
import numpy.random as npr
import pandas as pd
import random
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.extmath import cartesian
from bs4 import BeautifulSoup
import requests
import csv
import mechanize 
import time
import re
from collections import defaultdict

#r = requests.get('http://www.listingallcars.com/results/New')
#data = r.text
#soup = BeautifulSoup(data)
#for link in sou
#p.find_all('a')



In [2]:
br = mechanize.Browser(factory=mechanize.RobustFactory())
url = 'http://www.espn.com/nba/playbyplay?gameId=400900084'
# #url = 'http://www.everycarlisted.com/search/vehicle_condition-New/make-'+cartype+'/year-2017-2017/page-'
br.open(url)
response = br.response()
soup = BeautifulSoup(response,"html.parser")
#print soup.prettify()

In [3]:
teams =  soup.find_all('span',{'class' : 'cscore_name cscore_name--long'})
home_team = teams[0].contents
away_team = teams[1].contents

In [4]:
###Helper functino for make or miss
def toBinary(string):
    if (string == 'made'):
        return 1
    else:
        return 0 

In [5]:
text_home_team = soup.find('ul', { "class" : "shots home-team" })
text_away_team = soup.find('ul', { "class" : "shots away-team" })
shot_home = []
shot_away = []
top_home = []
top_away = []
left_home = []
left_away = []
status_home = []
status_away = []
shooting_team_home = []
shooting_team_away = []

for x in text_home_team.find_all('li'):
    status = str(x.get('class')[0])
    text = ('{} {}'.format(x.get('id'), x.get('style').split(';')[-3:-1]))
    text =text.split()
    shot_value = (re.search(r'\d+', text[0][4:]).group())
    left_value = float((re.search(r'\d+', text[1][4:]).group()))/100
    top_value = float((re.search(r'\d+', text[2][4:]).group()))/100
    shot_home.append(shot_value)
    top_home.append(top_value)
    left_home.append(left_value)
    status_home.append(toBinary(status))

for x in text_away_team.find_all('li'):
    status = str(x.get('class')[0])
    text = ('{} {}'.format(x.get('id'), x.get('style').split(';')[-3:-1]))
    text =text.split()
    shot_value = (re.search(r'\d+', text[0][4:]).group())
    left_value = float((re.search(r'\d+', text[1][4:]).group()))/100
    top_value = float((re.search(r'\d+', text[2][4:]).group()))/100
    shot_away.append(shot_value)
    top_away.append(top_value)
    left_away.append(left_value)
    status_away.append(toBinary(status))

shot_home = np.array(shot_home)
top_home = np.array(top_home)
left_home = np.array(left_home)
status_home = np.array(status_home)

shot_away = np.array(shot_away)
top_away = np.array(top_away)
left_away = np.array(left_away)
status_away = np.array(status_away)

# print shot_home.shape
# print top_home.shape
# print left_home.shape
# print shooting_team_home.shape
# print status_home.shape


# print len(shot_home)
# print len(top_home)
# print len(left_home)
# print len(shooting_team_home)
# print len(status_home)


HomeTeam = pd.DataFrame({'Shooting Team': home_team*len(shot_home),'Shot Number':shot_home,
                         'Left Position': left_home,'Top Position': top_home,'Home Team': home_team*(len(shot_home)),
                        'Away Team': away_team*len(shot_home),'Shot Status': status_home})




AwayTeam = pd.DataFrame({'Shooting Team': away_team*len(shot_away),'Shot Number':shot_away,
                         'Left Position': left_away,'Top Position': top_away,'Home Team': home_team*(len(shot_away)),
                        'Away Team': away_team*len(shot_away),'Shot Status': status_away})
HomeTeam = HomeTeam[['Shooting Team','Shot Number','Left Position','Top Position','Home Team','Away Team','Shot Status']]
AwayTeam = AwayTeam[['Shooting Team','Shot Number','Left Position','Top Position','Home Team','Away Team','Shot Status']]

    
# print('{} {}'.format(x.get('id'), x.get('style').split(';')[-2]))    

#soup = BeautifulSoup(response,"html.parser")
# # for row in soup.find_all('div',{ "class" : "sansBold font14 title" }):
# #     carname.append(row.text)    
# # for row in soup.find_all('div',{ "class" : "col-sm-6 col-xs-6 price" }):
# #     price.append(row.text[8:])

In [ ]:
for x in text_home_team.find_all('li'):
    print str(x.get('class')[0])
    print toBinary(str(x.get('class')[0]))
    
    text = ('{} {}'.format(x.get('id'), x.get('style').split(';')[-3:-1]))
    print text
#     text =text.split()
#     shot_value = (re.search(r'\d+', text[0][4:]).group())
#     left_value = float((re.search(r'\d+', text[1][4:]).group()))/100
#     top_value = float((re.search(r'\d+', text[2][4:]).group()))/100
#     shot_home.append(shot_value)
#     top_home.append(top_value)
#     left_home.append(left_value)

In [6]:
print HomeTeam.head()

  Shooting Team Shot Number  Left Position  Top Position     Home Team  \
0  Phoenix Suns           0           0.90          0.32  Phoenix Suns   
1  Phoenix Suns           3           0.94          0.50  Phoenix Suns   
2  Phoenix Suns           4           0.94          0.56  Phoenix Suns   
3  Phoenix Suns           5           0.92          0.52  Phoenix Suns   
4  Phoenix Suns           6           0.92          0.48  Phoenix Suns   

         Away Team  Shot Status  
0  Toronto Raptors            1  
1  Toronto Raptors            1  
2  Toronto Raptors            0  
3  Toronto Raptors            1  
4  Toronto Raptors            1  


In [ ]:
`def url_change(car_make = None ,model = None , price_interval = None, body = None,colors = None):
    url = 'http://www.everycarlisted.com/search/vehicle_condition-New/year-2017-2017'
    if car_make != None:
        url = url +'/make-'+car_make
    if model != None:
        url = url + '/model-'+model
    if colors != None:
        url = url + '/extcolor-'+colors
    if body != None:
        url = url + '/body-'+body
    if price_interval != None:
        url = url + '/price-'+price_interval
    url = url + '/sort-1/page-'
    return url

####Browser mechanize functionality
br = mechanize.Browser(factory=mechanize.RobustFactory())
# cartype = 'Rolls-Royce'
# url = 'http://www.everycarlisted.com/search/vehicle_condition-New/make-'+cartype+'/year-2017-2017/page-'
# br.open(url+str(1))
# response = br.response()
# soup = BeautifulSoup(response,"html.parser")

def Open_Browser(url, page):
    br.open(url+str(page))
    response = br.response()
    soup = BeautifulSoup(response,"html.parser")
    return soup

def web2var(string):
    string = string.replace('+','')
    string = string.replace('-','')
    return string

def var2web(string):
    if ' ' in string:
        string = string.replace(' ','+')
    return string



In [ ]:
price = []
carname = []
# print key_list[11]
# print Car_Make[11]
# print modelDictionary[key_list[11]][0]

In [ ]:
####Browser mechanize functionality
br = mechanize.Browser(factory=mechanize.RobustFactory())
# cartype = 'Rolls-Royce'
# url = 'http://www.everycarlisted.com/search/vehicle_condition-New/make-'+cartype+'/year-2017-2017/page-'
# br.open(url+str(1))
# response = br.response()
# soup = BeautifulSoup(response,"html.parser")

def Open_Browser(url, page):
    br.open(url+str(page))
    response = br.response()
    soup = BeautifulSoup(response,"html.parser")
    return soup

#@Deprecated
# def Total_Trucks(soup, cartype, body=None): 
#     total_cars = str(soup.body.h1.text)
#     print total_cars
#     if body == 'suv' or body =='pickup':
#         total_cars = total_cars[0:len(total_cars)-32-len(cartype)].replace(',','')
#     else:
#         total_cars = total_cars[0:len(total_cars)-30-len(cartype)].replace(',','')
#     print total_cars
#     total_cars= int(total_cars)
#     return total_cars

def Total_Cars(soup, cartype):
    total_cars = str(soup.body.h1.text)
    print total_cars
    total_cars = total_cars.replace(',','')
    total_cars = int(re.search(r'\d+', total_cars).group())
#   total_cars = total_cars[0:len(total_cars)-30-len(cartype)].replace(',','')
    print total_cars
    total_cars= int(total_cars)
    return total_cars
    
#46010 is total
#i = 1
#haven't done Chevrolet because they have a lot (175k)
# i = 12 is cheverolet
#for i in range(12,len(Car_Make)):

for i in range(1,len(Car_Make)):
    url = url_change(Car_Make[i])
    soup = Open_Browser(url, i)
    total_cars = Total_Cars(soup, Car_Make[i])
    print url;
    #     print total_cars
    if total_cars > 3000:
        #model iteration
        for h in range(0 , len(modelDictionary[key_list[i]])):
            url = url_change(Car_Make[i], modelDictionary[key_list[i]][h])
            soup = Open_Browser(url, i)
            total_cars = Total_Cars(soup, Car_Make[i])
            print url
            print total_cars
            if total_cars > 3000:
                for k in range(0 , len(Price_Interval)):
                    url = url_change(Car_Make[i], modelDictionary[key_list[i]][h], Price_Interval[k])
                    soup = Open_Browser(url, i)
                    total_cars = Total_Cars(soup, Car_Make[i])
                    print url
                    print total_cars
                    if total_cars > 3000:
                        for l in range(0 , len(Body)):
                            url = url_change(Car_Make[i], modelDictionary[key_list[i]][h], Price_Interval[k],Body[l])
                            soup = Open_Browser(url, i)
                            total_cars = Total_Cars(soup, Car_Make[i])
                            print url
                            print total_cars
                            if total_cars > 3000:
                                for j in range(0,len(Colors)):
                                    url = url_change(Car_Make[i], modelDictionary[key_list[i]][h], Price_Interval[k],Body[l],Colors[j])
                                    soup = Open_Browser(url, i)
                                    total_cars = Total_Cars(soup, Car_Make[i])
                                    print url
                                    print total_cars
                            else:
                                for page in range(1,int(math.ceil(total_cars/30.0))+1):
                                    soup = Open_Browser(url,page)
                                    for row in soup.find_all('div',{ "class" : "sansBold font14 title" }):
                                        carname.append(row.text)    
                                    for row in soup.find_all('div',{ "class" : "col-sm-6 col-xs-6 price" }):
                                        price.append(row.text[8:])
                                    print page
                    else:
                        for page in range(1,int(math.ceil(total_cars/30.0))+1):
                            soup = Open_Browser(url,page)
                            for row in soup.find_all('div',{ "class" : "sansBold font14 title" }):
                                carname.append(row.text)    
                            for row in soup.find_all('div',{ "class" : "col-sm-6 col-xs-6 price" }):
                                price.append(row.text[8:])
                            print page  
            else:
                for page in range(1,int(math.ceil(total_cars/30.0))+1):
                    soup = Open_Browser(url,page)
                    for row in soup.find_all('div',{ "class" : "sansBold font14 title" }):
                        carname.append(row.text)    
                    for row in soup.find_all('div',{ "class" : "col-sm-6 col-xs-6 price" }):
                        price.append(row.text[8:])
                    print page
    else:
        for page in range(1,int(math.ceil(total_cars/30.0))+1):
            soup = Open_Browser(url,page)
            for row in soup.find_all('div',{ "class" : "sansBold font14 title" }):
                carname.append(row.text)    
            for row in soup.find_all('div',{ "class" : "col-sm-6 col-xs-6 price" }):
                price.append(row.text[8:])
            print page


print 'Done'
                            
#     count = count +1
#     url_change = url + str(i)
#     br.open(url_change)
#     response = br.response()
#     soup = BeautifulSoup(response,"html.parser")
#     for row in soup.find_all('div',{ "class" : "sansBold font14 title" }):
#         carname.append(row.text)    
#     for row in soup.find_all('div',{ "class" : "col-sm-6 col-xs-6 price" }):
#         price.append(row.text[8:])
    

In [ ]:
# ####Browser mechanize functionality
# br = mechanize.Browser(factory=mechanize.RobustFactory())


# #Used to count and figure out which ones are over 3000 even after Body type
# def Open_Browser(url, page):
#     br.open(url+str(page))
#     response = br.response()
#     soup = BeautifulSoup(response,"html.parser")
#     return soup

# def Total_Trucks(soup, cartype, body=None):
#     total_cars = str(soup.body.h1.text)
#     print total_cars
#     if body == 'suv' or body =='pickup':
#         total_cars = total_cars[0:len(total_cars)-32-len(cartype)].replace(',','')
#     else:
#         total_cars = total_cars[0:len(total_cars)-30-len(cartype)].replace(',','')
#     print total_cars
#     total_cars= int(total_cars)
#     return total_cars

# def Total_Cars(soup, cartype):
#     total_cars = str(soup.body.h1.text)
#     print total_cars
#     total_cars = total_cars[0:len(total_cars)-30-len(cartype)].replace(',','')
#     print total_cars
#     total_cars= int(total_cars)
#     return total_cars
    
# #46010 is total
# for i in range(1,len(Car_Make)):
#     url = url_change(Car_Make[i])
#     soup = Open_Browser(url, i)
#     total_cars = Total_Cars(soup, Car_Make[i])
#     print url;
#     print total_cars
#     if total_cars > 3000:
#         print total_cars > 3000
#         for j in range(1,len(Price_Interval)):
#             url = url_change(Car_Make[i], Price_Interval[j])
#             soup = Open_Browser(url, i)
#             total_cars = Total_Cars(soup, Car_Make[i])
#             print url
#             print total_cars
# #     else:
#             if total_cars > 3000:
#                 for k in range(1 , len(Colors)):
#                     url = url_change(Car_Make[i], Price_Interval[j], Colors[k])
#                     soup = Open_Browser(url, i)
#                     total_cars = Total_Cars(soup, Car_Make[i])
#                     print url
#                     print total_cars
# #             else:
#                     if total_cars > 3000:
#                         for l in range(1 , len(Body)):
#                             url = url_change(Car_Make[i], Price_Interval[j], Colors[k],Body[l])
#                             soup = Open_Browser(url, i)
#                             total_cars = Total_Trucks(soup, Car_Make[i], Body[l])
#                             print url
#                             print total_cars

                    

In [ ]:
#############################Write Dataframe to .csv
output = pd.DataFrame(price,carname)
#print(output)
output.to_csv('CarPriceDistribution.csv', sep = ',', encoding = 'utf-8' )

print len(price)
#print("true")